In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
ns = '{http://www.volby.cz/kv/}'

In [3]:
partaje = ['KDU-ČSL','KSČM','ČSSD','ODS','ANO','STAN','SPD','TOP 09','Piráti']
kvros = pd.read_csv('../scratch/kvros.csv', encoding='windows-1250', sep=';')

kvr = {}
for k in list(kvros.to_dict(orient='index').values()):
    if k['ZKRATKAO8'] in partaje:
        kvr[ str(k['KODZASTUP']) + '_' + str(k['COBVODU']) + '_' + str(k['POR_STR_HL']) ] = k['ZKRATKAO8']

In [4]:
data = pd.DataFrame()
for i in range(1, 172): #172
    r = requests.get('http://80.188.53.60/pls/kvmedia/vysledky_okrsky?davka=' + str(i))
    root = ET.fromstring(r.text)
    for okr in root.findall(ns + 'OKRSEK'):
        tmp = okr.attrib
        tmp.update(okr.find(ns + 'UCAST_OKRSEK').attrib)
        for part in okr.findall(ns + 'HLASY_OKRSEK'):
            pid = okr.get('KODZASTUP') + '_' + okr.get('CIS_OBVODU') + '_' + part.get('POR_STR_HLAS_LIST')
            if pid in kvr.keys():
                tmp.update({'HL_' + kvr[pid]: int(part.get('HLASY'))})
            else:
                tmp.update({'HL_Ostatní': int(part.get('HLASY'))})
                
        data = data.append(tmp, ignore_index=True)

In [5]:
data.fillna(0, inplace=True)

In [6]:
for col in data.columns:
    if col.startswith('HL_'):
        data[col] = data[col].astype('int')

In [7]:
data.to_csv('../scratch/okrsky_cisla.csv', encoding='utf-8', index=False)